In [15]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [2]:
df=pd.read_csv("C:/Users/Cemil Turhan/Desktop/MachineLearning_FinalProject/Numerical_train.csv", index_col=0)

In [4]:
target=df.SalePrice
logged_target=np.log(target)
df = df.drop(labels = 'SalePrice', axis=1).astype("float64")

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df, logged_target)

# Hyperparameter Tuning for Regression

### Grid Search for alpha in Lasso 

In [6]:
lasso=Lasso()
alpha=[0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 1]
param_grid = dict(alpha=alpha)
lasso_cv = GridSearchCV(estimator = lasso, param_grid = param_grid, cv=5)
lasso_cv.fit(X_train, y_train)
lasso_cv.best_params_

C:\anac\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.792360140611056, tolerance: 0.014001908908737426
  positive)
C:\anac\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.19223449747506, tolerance: 0.014597434865364403
  positive)
C:\anac\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.137920558536963, tolerance: 0.014424321180966636
  positive)
C:\anac\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.327920999736285, tolerance: 0.014248806568614299
  pos

{'alpha': 0.0005}

### Grid Search for alpha in Ridge

In [7]:
ridge=Ridge()
alpha=[0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 3, 5, 10, 15, 20, 50, 100, 150, 200, 250]
param_grid = dict(alpha=alpha)
ridge_cv = GridSearchCV(estimator = ridge, param_grid = param_grid, cv=5)
ridge_cv.fit(X_train, y_train)
ridge_cv.best_params_

{'alpha': 5}

### Grid Search for Random Forest Parameters

In [8]:
rf=RandomForestRegressor()
n_estimators = [100, 300, 500, 700, 800, 900, 1000, 1200, 1350, 1500]
max_depth = [2, 5, 10, 15, 20, 25, 30, 40, 50]
min_samples_split = [1, 2, 4, 5, 8, 10]
min_samples_leaf = [2, 3, 4, 5, 8, 10]
param_grid = dict(n_estimators = n_estimators,
                  max_depth = max_depth,
                  min_samples_split = min_samples_split,
                  min_samples_leaf = min_samples_leaf)
rf_cv = GridSearchCV(estimator = rf, param_grid = param_grid, cv=5)
rf_cv.fit(X_train, y_train)
rf_cv.best_params_

C:\anac\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
C:\anac\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
C:\anac\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: min_samples_split must be an integer greater than 1 or a float in (0.0, 1.0]; got the integer 1

  FitFailedWarning)
C:\anac\lib\s

{'max_depth': 50,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 300}

In [9]:
best_params_rf={'max_depth': 50,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'n_estimators': 300}

### Regression Models with best parameters

In [11]:
def reg_models():
    lasso = Lasso(alpha=0.0005)
    ridge = Ridge(alpha=5)
    rf = RandomForestRegressor(max_depth=50, min_samples_leaf=2, min_samples_split=4, n_estimators=300)
    return lasso, ridge, rf

In [16]:
for name, model in zip(['lasso', 'ridge', 'rf'],
                       [*reg_models()]):
    if name == 'rf':
        pipeline = Pipeline([('scaler', None), ('estimator', model)])
    else:
        pipeline = Pipeline([('scaler', StandardScaler()), ('estimator', model)])
    scores = cross_validate(pipeline, X_train, y_train, cv=3, scoring=['r2', 'neg_mean_squared_error'])
    
    print(name, 'r2', scores['test_r2'].mean())
    print(name, 'rmse', np.sqrt(np.abs(scores['test_neg_mean_squared_error'])).mean())
    print('\n')

lasso r2 0.8609657225905695
lasso rmse 0.14949257939750363


ridge r2 0.8573490012422634
ridge rmse 0.1515597854157929


rf r2 0.8670042545503264
rf rmse 0.14696193255440204


